In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import random # we don't fix seed here so can use to re sample test patients

In [2]:
# if want to change global seed to sth else
seed = 0
from scvi._settings import settings
settings.seed = seed

Global seed set to 0
Global seed set to 0


# Download data

In [3]:
!wget --no-check-certificate https://covid19.cog.sanger.ac.uk/submissions/release1/haniffa21.processed.h5ad 

--2022-02-04 15:32:42--  https://covid19.cog.sanger.ac.uk/submissions/release1/haniffa21.processed.h5ad
Resolving covid19.cog.sanger.ac.uk (covid19.cog.sanger.ac.uk)... 193.62.203.61, 193.62.203.62, 193.62.203.63
Connecting to covid19.cog.sanger.ac.uk (covid19.cog.sanger.ac.uk)|193.62.203.61|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 7187322881 (6.7G) [application/x-hdf]
Saving to: 'haniffa21.processed.h5ad'

100%[====================================>] 7,187,322,881 10.1MB/s   in 7m 35s 

2022-02-04 15:40:17 (15.1 MB/s) - 'haniffa21.processed.h5ad' saved [7187322881/7187322881]



In [4]:
adata = sc.read('haniffa21.processed.h5ad')
adata

AnnData object with n_obs × n_vars = 647366 × 24929
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id'
    var: 'feature_types'
    uns: 'hvg', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'raw'

In [5]:
!wget https://static-content.springer.com/esm/art%3A10.1038%2Fs41591-021-01329-2/MediaObjects/41591_2021_1329_MOESM3_ESM.xlsx

--2022-02-04 15:40:32--  https://static-content.springer.com/esm/art%3A10.1038%2Fs41591-021-01329-2/MediaObjects/41591_2021_1329_MOESM3_ESM.xlsx
Resolving static-content.springer.com (static-content.springer.com)... 151.101.112.95
Connecting to static-content.springer.com (static-content.springer.com)|151.101.112.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12855042 (12M) [application/octet-stream]
Saving to: '41591_2021_1329_MOESM3_ESM.xlsx.1'

100%[======================================>] 12,855,042  --.-K/s   in 0.09s   

2022-02-04 15:40:32 (129 MB/s) - '41591_2021_1329_MOESM3_ESM.xlsx.1' saved [12855042/12855042]



In [6]:
dfs = pd.read_excel('41591_2021_1329_MOESM3_ESM.xlsx', sheet_name=None, header=1)
dfs.keys()

dict_keys(['Supp Table 1', 'Supp Table 2', 'Supp Table 3', 'Supp Table 4', 'Supp Table 5', 'Supp Table 6', 'Supp Table 7', 'Supp Table 8', 'Supp Table 9', 'Supp Table 10'])

In [7]:
df = dfs['Supp Table 2']
df

,Sample ID,Resample,Collection day,Patient ID,Age,Swab result,Status,Smoker,Status on collection day,Catergory on collection day,Status 3 days post collection,Status 7 days post collection,Days from onset,Site,Time after LPS,Worst clinical status,Outcome
0,BGCV01_CV0025,Initial,D0,CV0025,56,Negative,Covid,Non-smoker,Ward_O2,Moderate,Home,Home,16,Cambridge,NaN,Moderate,Home
1,BGCV06_CV0037,Initial,D0,CV0037,38,Positive,Covid,Non-smoker,Ward_O2,Moderate,Home,Home,12,Cambridge,NaN,Moderate,Home
2,BGCV11_CV0050,Initial,D0,CV0050,52,Positive,Covid,Non-smoker,Ward_noO2,Mild,Ward_noO2,Ward_noO2,11,Cambridge,NaN,Mild,Not_known
3,BGCV02_CV0052,Initial,D0,CV0052,83,Positive,Covid,Non-smoker,Ward_O2,Moderate,Death,Death,8,Cambridge,NaN,Death,Death
4,BGCV11_CV0058,Initial,D0,CV0058,49,Positive,Covid,Not_known,Staff screening,Asymptomatic,Staff screening,Staff screening,0,Cambridge,NaN,Asymptomatic,Home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,BGCV14_CV0284,Resample,D28,CV0284,63,Positive,Covid,Non-smoker,ITU_intubated,Critical,ITU_intubated,ITU_intubated,10,Cambridge,NaN,Critical,Not_known
139,newcastle004v2,Resample,D12,C-8821,75,Positive,Covid,Non-smoker,ITU_intubated,Critical,ITU_intubated,ITU_intubated,9,Ncl,NaN,Death,Death
140,newcastle21v2,Resample,D9,C-8884,71,Positive,Covid,Non-smoker,ITU_intubated,Critical,Ward_O2,Ward_O2,14,Ncl,NaN,Critical,Home
141,MH9143273,Resample,D7,C-8896,44,Positive,Covid,Non-smoker,ITU_intubated,Critical,ITU_intubated,ITU_intubated,6,Ncl,NaN,Critical,Home


# Clean data and merge annotations

In [8]:
samples = [sample.split('_')[-1] for sample in df['Sample ID']]

In [9]:
adata

AnnData object with n_obs × n_vars = 647366 × 24929
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id'
    var: 'feature_types'
    uns: 'hvg', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'raw'

In [10]:
# only take initial samples and either Covid or Healthy
adata = adata[adata.obs['Resample'] == 'Initial']
adata = adata[adata.obs['Status'].isin(['Covid', 'Healthy'])]
adata

/home/icb/anastasia.litinetskaya/miniconda3/envs/multi/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2631: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


View of AnnData object with n_obs × n_vars = 571327 × 24929
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id'
    var: 'feature_types'
    uns: 'hvg', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'raw'

In [11]:
assert(set(adata.obs['patient_id']).issubset(samples))

In [12]:
df.index = samples
df

,Sample ID,Resample,Collection day,Patient ID,Age,Swab result,Status,Smoker,Status on collection day,Catergory on collection day,Status 3 days post collection,Status 7 days post collection,Days from onset,Site,Time after LPS,Worst clinical status,Outcome
CV0025,BGCV01_CV0025,Initial,D0,CV0025,56,Negative,Covid,Non-smoker,Ward_O2,Moderate,Home,Home,16,Cambridge,NaN,Moderate,Home
CV0037,BGCV06_CV0037,Initial,D0,CV0037,38,Positive,Covid,Non-smoker,Ward_O2,Moderate,Home,Home,12,Cambridge,NaN,Moderate,Home
CV0050,BGCV11_CV0050,Initial,D0,CV0050,52,Positive,Covid,Non-smoker,Ward_noO2,Mild,Ward_noO2,Ward_noO2,11,Cambridge,NaN,Mild,Not_known
CV0052,BGCV02_CV0052,Initial,D0,CV0052,83,Positive,Covid,Non-smoker,Ward_O2,Moderate,Death,Death,8,Cambridge,NaN,Death,Death
CV0058,BGCV11_CV0058,Initial,D0,CV0058,49,Positive,Covid,Not_known,Staff screening,Asymptomatic,Staff screening,Staff screening,0,Cambridge,NaN,Asymptomatic,Home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CV0284,BGCV14_CV0284,Resample,D28,CV0284,63,Positive,Covid,Non-smoker,ITU_intubated,Critical,ITU_intubated,ITU_intubated,10,Cambridge,NaN,Critical,Not_known
newcastle004v2,newcastle004v2,Resample,D12,C-8821,75,Positive,Covid,Non-smoker,ITU_intubated,Critical,ITU_intubated,ITU_intubated,9,Ncl,NaN,Death,Death
newcastle21v2,newcastle21v2,Resample,D9,C-8884,71,Positive,Covid,Non-smoker,ITU_intubated,Critical,Ward_O2,Ward_O2,14,Ncl,NaN,Critical,Home
MH9143273,MH9143273,Resample,D7,C-8896,44,Positive,Covid,Non-smoker,ITU_intubated,Critical,ITU_intubated,ITU_intubated,6,Ncl,NaN,Critical,Home


In [13]:
ages = {}
for sample in set(adata.obs['patient_id']):
    ages[sample] = int(df['Age'].loc[[sample]][0])

In [14]:
ages = [ages[sample] for sample in adata.obs['patient_id']]
len(ages)

571327

In [15]:
adata.obs['Age'] = ages
adata.obs

Trying to set attribute `.obs` of view, copying.


,sample_id,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,full_clustering,initial_clustering,Resample,Collection_Day,...,Smoker,Status_on_day_collection,Status_on_day_collection_summary,Days_from_onset,Site,time_after_LPS,Worst_Clinical_Status,Outcome,patient_id,Age
covid_index,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGAAACCTA-MH9179824,MH9179824,1160,1160,2172.0,79.0,3.637201,CD8.TE,CD8,Initial,D0,...,Non-smoker,Ward_O2,Moderate,8,Ncl,nan,Moderate,Home,MH9179824,54
AAACCTGAGAGTAATC-MH9179824,MH9179824,1419,1419,3947.0,54.0,1.368128,CD4.IL22,CD4,Initial,D0,...,Non-smoker,Ward_O2,Moderate,8,Ncl,nan,Moderate,Home,MH9179824,54
AAACCTGAGAGTGAGA-MH9179824,MH9179824,1986,1986,5886.0,143.0,2.429494,CD8.Naive,CD4,Initial,D0,...,Non-smoker,Ward_O2,Moderate,8,Ncl,nan,Moderate,Home,MH9179824,54
AAACCTGAGGAATCGC-MH9179824,MH9179824,1508,1508,4271.0,106.0,2.481854,CD4.Naive,CD4,Initial,D0,...,Non-smoker,Ward_O2,Moderate,8,Ncl,nan,Moderate,Home,MH9179824,54
AAACCTGAGTGTTGAA-MH9179824,MH9179824,1109,1109,2279.0,76.0,3.334796,CD8.EM,CD4,Initial,D0,...,Non-smoker,Ward_O2,Moderate,8,Ncl,nan,Moderate,Home,MH9179824,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BGCV15_TTTCCTCTCTGATACG-1,BGCV15_CV0944,1811,1811,5821.0,80.0,1.374334,CD4.CM,CD4,Initial,D0,...,Not_known,Healthy,Healthy,Healthy,Cambridge,nan,Healthy,Home,CV0944,58
BGCV15_TTTCCTCTCTTTAGGG-1,BGCV15_CV0944,2097,2097,7480.0,138.0,1.844920,CD4.IL22,CD4,Initial,D0,...,Not_known,Healthy,Healthy,Healthy,Cambridge,nan,Healthy,Home,CV0944,58
BGCV15_TTTGCGCTCACCGTAA-1,BGCV15_CV0944,2031,2031,5257.0,73.0,1.388625,NK_16hi,NK_16hi,Initial,D0,...,Not_known,Healthy,Healthy,Healthy,Cambridge,nan,Healthy,Home,CV0944,58


# Split into train and test

In [16]:
adata

AnnData object with n_obs × n_vars = 571327 × 24929
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id', 'Age'
    var: 'feature_types'
    uns: 'hvg', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'raw'

In [17]:
len(adata.obs['patient_id'].cat.categories)

109

In [31]:
patient = 'patient_id'
label = 'Status_on_day_collection_summary'
number_of_test_patients = 10
patients = list(adata.obs[patient].cat.categories)
test_patient_ids = random.sample(patients, number_of_test_patients)
adata[adata.obs[patient].isin(test_patient_ids)].obs[[label, patient]].drop_duplicates().groupby(label).count() # check class distribution for test set

/home/icb/anastasia.litinetskaya/miniconda3/envs/multi/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2631: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


,patient_id
Status_on_day_collection_summary,
Asymptomatic,2
Critical,1
Healthy,2
Mild,1
Moderate,2
Severe,2


In [32]:
# to make sure that all of the classes are present in the test set
assert len(adata[adata.obs[patient].isin(test_patient_ids)].obs[label].unique()) == len(adata.obs[label].unique())

/home/icb/anastasia.litinetskaya/miniconda3/envs/multi/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2631: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [33]:
adata_test = adata[adata.obs[patient].isin(test_patient_ids)].copy()
adata_train = adata[~adata.obs[patient].isin(test_patient_ids)].copy()
adata_test.obs['split'] = "test"
adata_train.obs['split'] = "train"

In [34]:
adata = adata_train.concatenate(adata_test, index_unique=None)
adata

AnnData object with n_obs × n_vars = 571327 × 24929
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id', 'Age', 'split', 'batch'
    var: 'feature_types'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'raw'

In [35]:
rna = adata[:, adata.var['feature_types'] == 'Gene Expression']
adt = adata[:, adata.var['feature_types'] == 'Antibody Capture']

In [36]:
del adata

# RNA

In [38]:
rna = rna.copy()

In [39]:
rna.X = rna.layers['raw']

In [40]:
del rna.layers['raw']
rna

AnnData object with n_obs × n_vars = 571327 × 24737
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id', 'Age', 'split', 'batch'
    var: 'feature_types'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'

In [41]:
rna.layers['count'] = rna.X.copy()

In [42]:
sc.pp.normalize_total(rna, target_sum=1e4)
sc.pp.log1p(rna)

In [43]:
rna.write('stephenson2021_rna.h5ad')

... storing 'sample_id' as categorical
... storing 'full_clustering' as categorical
... storing 'Age_interval' as categorical
... storing 'Swab_result' as categorical
... storing 'Smoker' as categorical
... storing 'Status_on_day_collection' as categorical
... storing 'Days_from_onset' as categorical
... storing 'Site' as categorical
... storing 'Worst_Clinical_Status' as categorical
... storing 'Outcome' as categorical
... storing 'patient_id' as categorical
... storing 'split' as categorical


In [44]:
sc.pp.highly_variable_genes(rna, n_top_genes=4000)
rna = rna[:,rna.var.highly_variable]
rna

View of AnnData object with n_obs × n_vars = 571327 × 4000
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id', 'Age', 'split', 'batch'
    var: 'feature_types', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'count'

In [45]:
rna.write('stephenson2021_rna_hvg.h5ad')

# ADT

In [46]:
def clr_normalize_each_cell(adata, inplace=True):
    """Normalize count vector for each cell, i.e. for each row of .X"""

    import numpy as np
    import scipy

    def seurat_clr(x):
        s = np.sum(np.log1p(x[x > 0]))
        exp = np.exp(s / len(x))
        return np.log1p(x / exp)

    if not inplace:
        adata = adata.copy()

    # apply to dense or sparse matrix, along axis. returns dense matrix
    adata.X = np.apply_along_axis(
        seurat_clr, 1, (adata.X.A if scipy.sparse.issparse(adata.X) else adata.X)
    )
    return adata

In [48]:
adt = adt.copy()
adt.X = adt.layers['raw']
del adt.layers['raw']
adt

AnnData object with n_obs × n_vars = 571327 × 192
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id', 'Age', 'split', 'batch'
    var: 'feature_types'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'

In [49]:
adt.layers['count'] = adt.X.copy()

In [50]:
adt = clr_normalize_each_cell(adt)
adt

AnnData object with n_obs × n_vars = 571327 × 192
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id', 'Age', 'split', 'batch'
    var: 'feature_types'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'count'

In [52]:
adt.write('stephenson2021_adt.h5ad')

... storing 'sample_id' as categorical
... storing 'full_clustering' as categorical
... storing 'Age_interval' as categorical
... storing 'Swab_result' as categorical
... storing 'Smoker' as categorical
... storing 'Status_on_day_collection' as categorical
... storing 'Days_from_onset' as categorical
... storing 'Site' as categorical
... storing 'Worst_Clinical_Status' as categorical
... storing 'Outcome' as categorical
... storing 'patient_id' as categorical
... storing 'split' as categorical


# Checks

Check that raw counts are indeed in `.layers["count"]` and normalized counts in `.X`.

In [53]:
np.max(adt.X)

11.1271305

In [54]:
np.max(adt.layers['count'])

275279.0

In [55]:
np.max(rna.X)

9.070676

In [56]:
np.max(rna.layers['count'])

47487.0